In [ ]:
import os
import sys
import pandas as pd
import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pytz
sys.path.append('..')
import funcs.ac_funcs as ac

## For making sure met data doesnt have nans and or gaps

In [ ]:
met_folder = '/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/Data/met/wbb/daily_txt_gggformat'
ha_folder = '/uufs/chpc.utah.edu/common/home/lin-group9/agm/EM27/ha/inst_data/ifgs'

dates = os.listdir(ha_folder)
met_files = os.listdir(met_folder)

for date in dates:
    for met_file in met_files:
        if met_file.startswith(date):
            df = pd.read_csv(os.path.join(met_folder,met_file))
            df['dt'] = pd.to_datetime(df['UTCDate'] + ' ' + df['UTCTime'], format='%y/%m/%d %H:%M:%S')
            df.set_index('dt',inplace = True)
            df['timediff'] = df.index.diff()
            gaps_locations = df[df['timediff'] > pd.Timedelta(minutes=5)].index
            if len(gaps_locations) == 0:
                continue
            else:
                print('gap',gaps_locations)



In [ ]:
for met_file in met_files:
    df = pd.read_csv(os.path.join(met_folder,met_file))
    if len(df[df.isnull().any(axis=1)]) > 0 :
        print(df[df.isnull().any(axis=1)])

## Comparing met data

In [ ]:
dt1_str = '2024-07-16 08:00:00' #Start Datetime
dt2_str =  '2024-07-17 00:00:00' #End Datetime

mlg = ac.met_loader_ggg('/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/Data/met/wbb/daily_txt_gggformat/')
dt1 = datetime.datetime.strptime(dt1_str,'%Y-%m-%d %H:%M:%S') #create the datetime
dt1 = pytz.timezone('UTC').localize(dt1) #apply the timezone
dt2 = datetime.datetime.strptime(dt2_str,'%Y-%m-%d %H:%M:%S') #create the datetime
dt2 = pytz.timezone('UTC').localize(dt2) #apply the timezone

wbb_met_df = mlg.load_data_inrange(dt1,dt2)
wbb_met_df.index = wbb_met_df.index.tz_convert('US/Mountain')

mlg = ac.met_loader_ggg('/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/Data/met/vaisala/')
vaisala_met_df = mlg.load_data_inrange(dt1,dt2)
vaisala_met_df.index = vaisala_met_df.index.tz_convert('US/Mountain')

In [ ]:
params = ['pres','rh','temp']
df_dict = {'wbb':[wbb_met_df,'black'],'vaisala':[vaisala_met_df,'red']}

fig = make_subplots(rows=len(params),cols=1,shared_xaxes=True)

row = 1
for param in params:
    for key,values in df_dict.items():
        df = values[0]
        color = values[1]
        fig.add_trace(go.Scatter(
        x = df.index,
        y = df[param],
        mode = 'markers',
        marker = {'color' : color,'size':3},
        name = key,
        showlegend = True,
        ),row = row,col = 1)
        fig.update_yaxes(title_text = param,row=row,col=1)
    row+=1

fig.update_layout(
    height=600,
    margin = go.layout.Margin(t=1,b=1)
)
fig.show()